### 1) 데이터셋의 크기가 1/10일 때 , k-fold crossvalidation, gradient descent를 이용한 linear regression
### 2) 데이터셋의 크기가 원래 사이즈일때, k-fold crossvalidation, gradient descent를 이용한 linear regression 
### 3) 1과 2의 성능비교

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# 데이터 불러오기 
file = open('/content/drive/MyDrive/Lion/data/regression_data.txt','r')  # open the file with read-only
text = file.readlines()  # read all line texts
file.close()  # close the file

# 1/10
x_data = []
y_data = []

# original
x_origin_data = []
y_origin_data = []

# 데이터 1/10

# convert to float
for idx,s in enumerate(text):
    data = s.split()
    x_origin_data.append(float(data[0]))
    y_origin_data.append(float(data[1])) 
    
    if idx%10==0:
      x_data.append(float(data[0]))
      y_data.append(float(data[1]))    

# convert to numpy-array
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)
x_origin_data = np.asarray(x_origin_data, dtype=np.float32)
y_origin_data = np.asarray(y_origin_data, dtype=np.float32)

In [ ]:
print(len(x_origin_data), len(x_data))

100 10


In [ ]:
# data shaping for pytorch : (input.shape: (데이터의 갯수, 차원) )
if len(x_data.shape)==1 and len(y_data.shape)==1:
  x_data = np.expand_dims(x_data, axis=-1)
  y_data = np.expand_dims(y_data, axis=-1)

if len(x_origin_data.shape)==1 and len(y_origin_data.shape)==1:
  x_origin_data = np.expand_dims(x_origin_data, axis=-1)
  y_origin_data = np.expand_dims(y_origin_data, axis=-1)
print(x_origin_data.shape, y_origin_data.shape)

(100, 1) (100, 1)


In [ ]:
# k=np.setdiff1d(x_data,x_data[2:4] )
# k2=torch.from_numpy(k)
# k2.shape

torch.Size([8])

In [ ]:
# x_data[2:4,].shape

(2, 1)

성능비교

In [ ]:
# 1/10 data
# Hyper-parameters
input_size = 1
output_size = 1
num_epochs = 100
learning_rate = 0.1

k=5
validation_x_loss=[]

val_losses = []

for i in range(k):
  model = nn.Linear(input_size, output_size) 

  # Loss and optimizer
  criterion = nn.MSELoss()  
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

  x_valid=x_data[2*i:2*(i+1),]
  y_valid=y_data[2*i:2*(i+1),]
  
  x_train=np.setdiff1d(x_data,x_valid )
  y_train=np.setdiff1d(y_data,y_valid )

  if len(x_train.shape)==1 and len(y_train.shape)==1:
    x_train = np.expand_dims(x_train, axis=-1)
    y_train = np.expand_dims(y_train, axis=-1)


    # Train the model
  for epoch in range(num_epochs):
      # Convert numpy arrays to torch tensors
      inputs = torch.from_numpy(x_train)
      targets = torch.from_numpy(y_train)

      # Predict outputs with the linear model.
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      
      # compute gradients and update
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      if (epoch+1) % 20 == 0:
          print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

  # validation error
  inputs = torch.from_numpy(x_valid)
  targets = torch.from_numpy(y_valid)
  outputs = model(inputs)
  loss = criterion(outputs, targets)
  print(i+1,"-th round validation error: ",loss.item())
  val_losses.append(loss.item())

val_losses = np.asarray(val_losses)
print("Final validation error: ", val_losses.mean())

Epoch [20/100], Loss: 0.0204
Epoch [40/100], Loss: 0.0013
Epoch [60/100], Loss: 0.0003
Epoch [80/100], Loss: 0.0002
Epoch [100/100], Loss: 0.0002
1 -th round validation error:  8.724057988729328e-05
Epoch [20/100], Loss: 0.0132
Epoch [40/100], Loss: 0.0017
Epoch [60/100], Loss: 0.0004
Epoch [80/100], Loss: 0.0002
Epoch [100/100], Loss: 0.0002
2 -th round validation error:  1.3064389349892735e-05
Epoch [20/100], Loss: 0.0059
Epoch [40/100], Loss: 0.0006
Epoch [60/100], Loss: 0.0002
Epoch [80/100], Loss: 0.0001
Epoch [100/100], Loss: 0.0001
3 -th round validation error:  0.00043226982234045863
Epoch [20/100], Loss: 0.0165
Epoch [40/100], Loss: 0.0018
Epoch [60/100], Loss: 0.0003
Epoch [80/100], Loss: 0.0002
Epoch [100/100], Loss: 0.0002
4 -th round validation error:  0.0003998020547442138
Epoch [20/100], Loss: 0.0002
Epoch [40/100], Loss: 0.0001
Epoch [60/100], Loss: 0.0001
Epoch [80/100], Loss: 0.0001
Epoch [100/100], Loss: 0.0001
5 -th round validation error:  0.0007297149859368801
Fin

In [ ]:
# origin data

# Hyper-parameters
input_size = 1
output_size = 1
num_epochs = 100
learning_rate = 0.1

k=5
validation_x_loss=[]

val_losses = []

for i in range(k):
  model = nn.Linear(input_size, output_size) 

  # Loss and optimizer
  criterion = nn.MSELoss()  
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

  x_valid=x_origin_data[2*i:2*(i+1),]
  y_valid=y_origin_data[2*i:2*(i+1),]
  
  x_train=np.setdiff1d(x_origin_data,x_valid )
  y_train=np.setdiff1d(y_origin_data,y_valid )

  if len(x_train.shape)==1 and len(y_train.shape)==1:
    x_train = np.expand_dims(x_train, axis=-1)
    y_train = np.expand_dims(y_train, axis=-1)


    # Train the model
  for epoch in range(num_epochs):
      # Convert numpy arrays to torch tensors
      inputs = torch.from_numpy(x_train)
      targets = torch.from_numpy(y_train)

      # Predict outputs with the linear model.
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      
      # compute gradients and update
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      if (epoch+1) % 20 == 0:
          print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

  # validation error
  inputs = torch.from_numpy(x_valid)
  targets = torch.from_numpy(y_valid)
  outputs = model(inputs)
  loss = criterion(outputs, targets)
  print(i+1,"-th round validation error: ",loss.item())
  val_losses.append(loss.item())

val_losses = np.asarray(val_losses)
print("Final validation error: ", val_losses.mean())

Epoch [20/100], Loss: 0.0232
Epoch [40/100], Loss: 0.0024
Epoch [60/100], Loss: 0.0003
Epoch [80/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
1 -th round validation error:  0.00017517681408207864
Epoch [20/100], Loss: 0.0020
Epoch [40/100], Loss: 0.0002
Epoch [60/100], Loss: 0.0000
Epoch [80/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
2 -th round validation error:  3.7420144508359954e-05
Epoch [20/100], Loss: 0.0015
Epoch [40/100], Loss: 0.0002
Epoch [60/100], Loss: 0.0000
Epoch [80/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
3 -th round validation error:  0.00010497155744815245
Epoch [20/100], Loss: 0.0029
Epoch [40/100], Loss: 0.0003
Epoch [60/100], Loss: 0.0000
Epoch [80/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
4 -th round validation error:  4.4390086259227246e-05
Epoch [20/100], Loss: 0.0009
Epoch [40/100], Loss: 0.0001
Epoch [60/100], Loss: 0.0000
Epoch [80/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
5 -th round validation error:  0.00016486275126226246


## 결론 : 테스트셋이 많을수록 성능이 좋다